<a href="https://colab.research.google.com/github/git4sudo/fin_rl_vs_llm/blob/main/stockemotions_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install swig
!pip install wrds
!pip install pyportfolioopt
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-g7pdqt45
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-g7pdqt45
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d2b0d88a71e8058646d41c2eb0eb29d70e258997
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-fe0soosi/elegantrl_f930cef4b7b94ce0a944006aa102feea
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-fe0soosi/elegantrl_f930cef4b7b94ce0a944006aa102feea
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 155f07fcfe2d0f0a0318f820e8e2f2401ff30eca
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [ ]:
stockemotions=pd.read_csv('/content/stockemotion_merged.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,id,date,ticker,emo_label,senti_label,original,processed
0,0,100001,2020-01-01,AMZN,excitement,bullish,$AMZN Dow futures up by 100 points already 🥳,Amazon Dow futures up by 100 points already [...
1,1,100002,2020-01-01,TSLA,excitement,bullish,$TSLA Daddy's drinkin' eArly tonight! Here's t...,Tesla Daddy's drinkin' eArly tonight! Here's t...
2,2,100003,2020-01-01,AAPL,confusion,bullish,$AAPL We’ll been riding since last December fr...,Apple We’ll been riding since last December fr...
3,3,100004,2020-01-01,TSLA,excitement,bullish,"$TSLA happy new year, 2020, everyone🍷🎉🙏","Tesla happy new year, 2020, everyone [wine gla..."
4,4,100005,2020-01-01,TSLA,excitement,bullish,"$TSLA haha just a collection of greats...""Mars...","Tesla haha just a collection of greats...""Mars..."
...,...,...,...,...,...,...,...,...
9995,995,109996,2020-12-31,ABNB,optimism,bearish,"$ABNB “sugar daddy puts.” Don’t mind me, I’m j...","Airbnb “sugar daddy puts.” Don’t mind me, I’m ..."
9996,996,109997,2020-12-31,TSLA,disgust,bullish,$TSLA \nGood news... now bears can get help wh...,Tesla \nGood news... now bears can get help wh...
9997,997,109998,2020-12-31,BABA,confusion,bullish,$BABA Who else is glad they sold in 240s yeste...,Alibaba Who else is glad they sold in 240s yes...
9998,998,109999,2020-12-31,CCL,amusement,bullish,$CCL $23 calls for .79 you know what to do 🥳,Carnival $23 calls for .79 you know what to do...


In [ ]:
stockemotions.drop(columns=['Unnamed: 0','id','original','processed'], inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions.rename(columns={'ticker': 'tic'}, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions = stockemotions.drop_duplicates(subset=['date', 'tic'])

In [ ]:
stockemotions

,date,tic,emo_label,senti_label
0,2020-01-01,AMZN,excitement,bullish
1,2020-01-01,TSLA,excitement,bullish
2,2020-01-01,AAPL,confusion,bullish
6,2020-01-02,AAPL,amusement,bullish
9,2020-01-02,HD,optimism,bullish
...,...,...,...,...
9969,2020-12-18,FB,panic,bearish
9984,2020-12-24,GOOG,optimism,bullish
9987,2020-12-27,AAPL,optimism,bullish
9989,2020-12-29,PFE,confusion,bearish


In [ ]:
stockemotions['tic'] = stockemotions['tic'].str.lower()
Stocks=['aapl','ba','amzn','dis','msft','ccl','nvda','wmt','bac','sbux']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions = stockemotions[stockemotions['tic'].isin(Stocks)]

In [ ]:
stockemotions

,date,tic,emo_label,senti_label
0,2020-01-01,amzn,excitement,bullish
2,2020-01-01,aapl,confusion,bullish
6,2020-01-02,aapl,amusement,bullish
13,2020-01-03,nvda,belief,bullish
14,2020-01-03,aapl,optimism,bullish
...,...,...,...,...
9911,2020-11-26,ccl,disgust,bullish
9960,2020-12-15,ba,anger,bearish
9968,2020-12-18,amzn,amusement,bearish
9987,2020-12-27,aapl,optimism,bullish


In [ ]:
stockemotions.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Index: 1296 entries, 0 to 9998
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1296 non-null   object
 1   tic          1296 non-null   object
 2   emo_label    1296 non-null   object
 3   senti_label  1296 non-null   object
dtypes: object(4)
memory usage: 82.9+ KB


In [ ]:
stockemotions_by_dates = stockemotions.set_index(stockemotions.columns[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
grouped = stockemotions.groupby('tic')

In [ ]:
ticker_names = stockemotions['tic'].unique().tolist()

In [ ]:
dfs = {ticker: pd.DataFrame() for ticker in ticker_names}

In [ ]:
for ticker, group in grouped:
    if ticker in dfs:
        dfs[ticker] = group

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
shapes_dict = {ticker: df.shape[0] for ticker, df in dfs.items()}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sorted_shapes = sorted(shapes_dict.items(), key=lambda item: item[1], reverse=True)

In [ ]:
for ticker, shape in sorted_shapes:
    print(f"{ticker}: {shape}")

aapl: 296
ba: 211
amzn: 176
dis: 134
msft: 134
ccl: 115
nvda: 81
wmt: 70
bac: 43
sbux: 36


finrl data download

In [ ]:
TRAIN_START_DATE = '2000-01-01'
TRAIN_END_DATE = '2020-01-01'
TRADE_START_DATE = '2020-01-01'
TRADE_END_DATE = '2024-01-01'

In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = Stocks).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (60370, 8)


In [ ]:
df_raw

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day
0,2000-01-03,0.936384,1.004464,0.907924,0.844981,535796800,aapl,0
1,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000,amzn,0
2,2000-01-03,41.437500,41.687500,39.812500,25.940277,2638200,ba,0
3,2000-01-03,25.125000,25.125000,24.000000,12.946365,13705800,bac,0
4,2000-01-03,46.000000,47.125000,45.250000,28.774662,1532600,ccl,0
...,...,...,...,...,...,...,...,...
60365,2023-12-29,90.120003,90.599998,89.860001,90.290001,9201300,dis,4
60366,2023-12-29,376.000000,377.160004,373.480011,375.345886,18723000,msft,4
60367,2023-12-29,498.130005,499.970001,487.510010,495.196777,38869000,nvda,4
60368,2023-12-29,95.879997,96.349998,95.349998,95.435814,6134000,sbux,4


In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Successfully added technical indicators


[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (6036, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
processed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2000-01-03,0.936384,1.004464,0.907924,0.844981,535796800,aapl,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,0.844981,0.844981,24.209999,0.000000
1,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000,amzn,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,4.468750,4.468750,24.209999,0.000000
2,2000-01-03,41.437500,41.687500,39.812500,25.940277,2638200,ba,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,25.940277,25.940277,24.209999,0.000000
3,2000-01-03,25.125000,25.125000,24.000000,12.946365,13705800,bac,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,12.946365,12.946365,24.209999,0.000000
4,2000-01-03,46.000000,47.125000,45.250000,28.774662,1532600,ccl,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,28.774662,28.774662,24.209999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60355,2023-12-28,90.089996,90.980003,89.970001,90.400002,8479600,dis,3,0.245315,94.192660,89.735047,51.717656,-131.472305,1.737710,92.652751,88.158208,12.470000,0.773413
60356,2023-12-28,375.369995,376.459991,374.160004,374.587280,14327000,msft,3,2.668700,377.711291,366.355744,58.167711,44.673886,23.578481,373.178901,355.870528,12.470000,0.773413
60357,2023-12-28,496.429993,498.839996,494.119995,495.196777,24658700,nvda,3,6.927852,507.264027,451.336782,56.171621,87.376634,21.097066,482.416309,463.570170,12.470000,0.773413
60358,2023-12-28,95.500000,95.970001,95.110001,95.356293,4518300,sbux,3,-1.057010,98.969987,93.588490,46.833962,-74.817933,25.099002,98.526552,96.783740,12.470000,0.773413


In [ ]:
processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60360 entries, 0 to 60359
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          60360 non-null  object 
 1   open          60360 non-null  float64
 2   high          60360 non-null  float64
 3   low           60360 non-null  float64
 4   close         60360 non-null  float64
 5   volume        60360 non-null  int64  
 6   tic           60360 non-null  object 
 7   day           60360 non-null  int32  
 8   macd          60360 non-null  float64
 9   boll_ub       60360 non-null  float64
 10  boll_lb       60360 non-null  float64
 11  rsi_30        60360 non-null  float64
 12  cci_30        60360 non-null  float64
 13  dx_30         60360 non-null  float64
 14  close_30_sma  60360 non-null  float64
 15  close_60_sma  60360 non-null  float64
 16  vix           60360 non-null  float64
 17  turbulence    60360 non-null  float64
dtypes: float64(14), int32(1), 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions['date'] = pd.to_datetime(stockemotions['date'])
processed['date'] = pd.to_datetime(processed['date'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2000-01-03,0.936384,1.004464,0.907924,0.844981,535796800,aapl,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,0.844981,0.844981,24.209999,0.000000
1,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000,amzn,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,4.468750,4.468750,24.209999,0.000000
2,2000-01-03,41.437500,41.687500,39.812500,25.940277,2638200,ba,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,25.940277,25.940277,24.209999,0.000000
3,2000-01-03,25.125000,25.125000,24.000000,12.946365,13705800,bac,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,12.946365,12.946365,24.209999,0.000000
4,2000-01-03,46.000000,47.125000,45.250000,28.774662,1532600,ccl,0,0.000000,0.910109,0.708612,0.000000,-66.666667,100.000000,28.774662,28.774662,24.209999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60355,2023-12-28,90.089996,90.980003,89.970001,90.400002,8479600,dis,3,0.245315,94.192660,89.735047,51.717656,-131.472305,1.737710,92.652751,88.158208,12.470000,0.773413
60356,2023-12-28,375.369995,376.459991,374.160004,374.587280,14327000,msft,3,2.668700,377.711291,366.355744,58.167711,44.673886,23.578481,373.178901,355.870528,12.470000,0.773413
60357,2023-12-28,496.429993,498.839996,494.119995,495.196777,24658700,nvda,3,6.927852,507.264027,451.336782,56.171621,87.376634,21.097066,482.416309,463.570170,12.470000,0.773413
60358,2023-12-28,95.500000,95.970001,95.110001,95.356293,4518300,sbux,3,-1.057010,98.969987,93.588490,46.833962,-74.817933,25.099002,98.526552,96.783740,12.470000,0.773413


In [ ]:
processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60360 entries, 0 to 60359
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          60360 non-null  datetime64[ns]
 1   open          60360 non-null  float64       
 2   high          60360 non-null  float64       
 3   low           60360 non-null  float64       
 4   close         60360 non-null  float64       
 5   volume        60360 non-null  int64         
 6   tic           60360 non-null  object        
 7   day           60360 non-null  int32         
 8   macd          60360 non-null  float64       
 9   boll_ub       60360 non-null  float64       
 10  boll_lb       60360 non-null  float64       
 11  rsi_30        60360 non-null  float64       
 12  cci_30        60360 non-null  float64       
 13  dx_30         60360 non-null  float64       
 14  close_30_sma  60360 non-null  float64       
 15  close_60_sma  60360 non-null  float6

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed = processed.merge(stockemotions[['date', 'tic', 'emo_label', 'senti_label']],
                            on=['date', 'tic'],
                            how='left')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60360 entries, 0 to 60359
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          60360 non-null  datetime64[ns]
 1   open          60360 non-null  float64       
 2   high          60360 non-null  float64       
 3   low           60360 non-null  float64       
 4   close         60360 non-null  float64       
 5   volume        60360 non-null  int64         
 6   tic           60360 non-null  object        
 7   day           60360 non-null  int32         
 8   macd          60360 non-null  float64       
 9   boll_ub       60360 non-null  float64       
 10  boll_lb       60360 non-null  float64       
 11  rsi_30        60360 non-null  float64       
 12  cci_30        60360 non-null  float64       
 13  dx_30         60360 non-null  float64       
 14  close_30_sma  60360 non-null  float64       
 15  close_60_sma  60360 non-null  float6

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ML data imputation

In [ ]:
!pip install tensorflow

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
scaler = StandardScaler()
processed[['open', 'high', 'low', 'close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'vix', 'turbulence']] = \
    scaler.fit_transform(processed[['open', 'high', 'low', 'close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'vix', 'turbulence']])

/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):


In [ ]:
processed_copy = processed.copy()
processed = processed.dropna(subset=['emo_label', 'senti_label'])
le_emo = LabelEncoder()
le_senti = LabelEncoder()
processed['emo_label'] = le_emo.fit_transform(processed['emo_label'].astype(str))
processed['senti_label'] = le_senti.fit_transform(processed['senti_label'].astype(str))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):


In [ ]:
num_classes_emo = np.max(processed['emo_label']) + 1
num_classes_senti = np.max(processed['senti_label']) + 1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
num_classes_emo

12

In [ ]:
num_classes_senti

2

In [ ]:
features = processed[['open', 'high', 'low', 'close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'vix', 'turbulence']].values
emo_labels = to_categorical(processed['emo_label'], num_classes=num_classes_emo)
senti_labels = to_categorical(processed['senti_label'], num_classes=num_classes_senti)

In [ ]:
features = np.reshape(features, (features.shape[0], 1, features.shape[1]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
X_train, X_test, y_train_emo, y_test_emo, y_train_senti, y_test_senti = train_test_split(
    features, emo_labels, senti_labels, test_size=0.2, random_state=810)

In [ ]:
model_emo = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dropout(0.2),
    Dense(num_classes_emo, activation='softmax')  # Output layer for emotion labels
])

In [ ]:
model_senti = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dropout(0.2),
    Dense(num_classes_senti, activation='softmax')  # Output layer for sentiment labels
])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
model_emo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_emo.fit(X_train, y_train_emo, epochs=25, batch_size=32, validation_data=(X_test, y_test_emo), callbacks=[early_stopping])

Epoch 1/25
30/30 [==============================] - 19s 74ms/step - loss: 2.4487 - accuracy: 0.1546 - val_loss: 2.4100 - val_accuracy: 0.2085
Epoch 2/25
30/30 [==============================] - 0s 10ms/step - loss: 2.3636 - accuracy: 0.1908 - val_loss: 2.3613 - val_accuracy: 0.1872
Epoch 3/25
30/30 [==============================] - 0s 10ms/step - loss: 2.3268 - accuracy: 0.1855 - val_loss: 2.3469 - val_accuracy: 0.2170
Epoch 4/25
30/30 [==============================] - 0s 9ms/step - loss: 2.3047 - accuracy: 0.2068 - val_loss: 2.3323 - val_accuracy: 0.2213
Epoch 5/25
30/30 [==============================] - 0s 10ms/step - loss: 2.2939 - accuracy: 0.1972 - val_loss: 2.3301 - val_accuracy: 0.2255
Epoch 6/25
30/30 [==============================] - 0s 9ms/step - loss: 2.2864 - accuracy: 0.2026 - val_loss: 2.3167 - val_accuracy: 0.2255
Epoch 7/25
30/30 [==============================] - 0s 11ms/step - loss: 2.2840 - accuracy: 0.2004 - val_loss: 2.3111 - val_accuracy: 0.2255
Epoch 8/25
30/

In [ ]:
model_senti.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_senti.fit(X_train, y_train_senti, epochs=25, batch_size=32, validation_data=(X_test, y_test_senti), callbacks=[early_stopping])

Epoch 1/25
30/30 [==============================] - 14s 72ms/step - loss: 0.6798 - accuracy: 0.6045 - val_loss: 0.6674 - val_accuracy: 0.6383
Epoch 2/25
30/30 [==============================] - 0s 8ms/step - loss: 0.6589 - accuracy: 0.6013 - val_loss: 0.6540 - val_accuracy: 0.6468
Epoch 3/25
30/30 [==============================] - 0s 9ms/step - loss: 0.6445 - accuracy: 0.6183 - val_loss: 0.6463 - val_accuracy: 0.6383
Epoch 4/25
30/30 [==============================] - 0s 8ms/step - loss: 0.6387 - accuracy: 0.6162 - val_loss: 0.6385 - val_accuracy: 0.6596
Epoch 5/25
30/30 [==============================] - 0s 9ms/step - loss: 0.6316 - accuracy: 0.6183 - val_loss: 0.6317 - val_accuracy: 0.6766
Epoch 6/25
30/30 [==============================] - 0s 9ms/step - loss: 0.6296 - accuracy: 0.6407 - val_loss: 0.6262 - val_accuracy: 0.6681
Epoch 7/25
30/30 [==============================] - 0s 10ms/step - loss: 0.6268 - accuracy: 0.6493 - val_loss: 0.6251 - val_accuracy: 0.6596
Epoch 8/25
30/30 

In [ ]:
loss_emo, accuracy_emo = model_emo.evaluate(X_test, y_test_emo)

8/8 [==============================] - 0s 4ms/step - loss: 2.3002 - accuracy: 0.2043


In [ ]:
loss_senti, accuracy_senti = model_senti.evaluate(X_test, y_test_senti)

8/8 [==============================] - 0s 4ms/step - loss: 0.6165 - accuracy: 0.6681


In [ ]:
print("Emotion Classification Accuracy:", accuracy_emo)

Emotion Classification Accuracy: 0.20425531268119812


In [ ]:
print("Sentiment Classification Accuracy:", accuracy_senti)

Sentiment Classification Accuracy: 0.6680850982666016


In [ ]:
missing_emo_indices = processed_copy['emo_label'].isna()
missing_senti_indices = processed_copy['senti_label'].isna()

In [ ]:
missing_emo_features = processed_copy.loc[missing_emo_indices, ['open', 'high', 'low', 'close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'vix', 'turbulence']]
missing_senti_features = processed_copy.loc[missing_senti_indices, ['open', 'high', 'low', 'close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'vix', 'turbulence']]

In [ ]:
if not missing_emo_features.empty:
    missing_emo_features = np.reshape(missing_emo_features.values, (missing_emo_features.shape[0], 1, missing_emo_features.shape[1]))
    predicted_emo = model_emo.predict(missing_emo_features)
    predicted_emo_labels = np.argmax(predicted_emo, axis=1)
    processed_copy.loc[missing_emo_indices, 'emo_label'] = le_emo.inverse_transform(predicted_emo_labels)

if not missing_senti_features.empty:
    missing_senti_features = np.reshape(missing_senti_features.values, (missing_senti_features.shape[0], 1, missing_senti_features.shape[1]))
    predicted_senti = model_senti.predict(missing_senti_features)
    predicted_senti_labels = np.argmax(predicted_senti, axis=1)
    processed_copy.loc[missing_senti_indices, 'senti_label'] = le_senti.inverse_transform(predicted_senti_labels)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1850/1850 [==============================] - 8s 3ms/step


In [ ]:
processed['emo_label'] = processed_copy['emo_label'].astype(str)
processed['senti_label'] = processed_copy['senti_label'].astype(str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_copy.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,emo_label,senti_label
0,2000-01-03,-0.782505,-0.781243,-0.783308,-0.714637,2.644707,aapl,0,-0.088967,-0.711177,-0.721376,-6.081124,-0.750238,4.649049,0.844981,0.844981,0.485454,-0.669293,disgust,bearish
1,2000-01-03,-0.734109,-0.728316,-0.735768,-0.658409,1.409771,amzn,0,-0.088967,-0.711177,-0.721376,-6.081124,-0.750238,4.649049,4.468750,4.468750,0.485454,-0.669293,disgust,bearish
2,2000-01-03,-0.157996,-0.161363,-0.175805,-0.325245,-0.440011,ba,0,-0.088967,-0.711177,-0.721376,-6.081124,-0.750238,4.649049,25.940277,25.940277,0.485454,-0.669293,disgust,bearish
3,2000-01-03,-0.409527,-0.413723,-0.422720,-0.526866,-0.375977,bac,0,-0.088967,-0.711177,-0.721376,-6.081124,-0.750238,4.649049,12.946365,12.946365,0.485454,-0.669293,optimism,bearish
4,2000-01-03,-0.087645,-0.078512,-0.090897,-0.281266,-0.446408,ccl,0,-0.088967,-0.711177,-0.721376,-6.081124,-0.750238,4.649049,28.774662,28.774662,0.485454,-0.669293,disgust,bearish


In [ ]:
# Assuming 'processed' is your DataFrame

# Get counts of unique values for 'emo_label'
emo_label_counts = processed_copy['emo_label'].value_counts()
print("Emotion Label Counts:\n", emo_label_counts)

# Get counts of unique values for 'senti_label'
senti_label_counts = processed_copy['senti_label'].value_counts()
print("Sentiment Label Counts:\n", senti_label_counts)


Emotion Label Counts:
 emo_label
optimism      53023
anxiety        4580
disgust         966
depression      624
anger           258
belief          241
panic           151
excitement      133
confusion       113
ambiguous       108
amusement        94
surprise         69
Name: count, dtype: int64
Sentiment Label Counts:
 senti_label
bullish    34205
bearish    26155
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stockemotions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,emo_label,senti_label
0,2020-01-01,amzn,excitement,bullish
2,2020-01-01,aapl,confusion,bullish
6,2020-01-02,aapl,amusement,bullish
13,2020-01-03,nvda,belief,bullish
14,2020-01-03,aapl,optimism,bullish
...,...,...,...,...
9911,2020-11-26,ccl,disgust,bullish
9960,2020-12-15,ba,anger,bearish
9968,2020-12-18,amzn,amusement,bearish
9987,2020-12-27,aapl,optimism,bullish


In [ ]:
emo_label_countsstockemotions = stockemotions['emo_label'].value_counts()
print("Emotion Label Counts:\n", emo_label_countsstockemotions)

# Get counts of unique values for 'senti_label'
senti_label_countsstockemotions = stockemotions['senti_label'].value_counts()
print("Sentiment Label Counts:\n", senti_label_countsstockemotions)

Emotion Label Counts:
 emo_label
optimism      250
anxiety       231
belief        143
excitement    130
disgust       107
amusement      99
confusion      92
ambiguous      92
anger          44
surprise       39
panic          37
depression     32
Name: count, dtype: int64
Sentiment Label Counts:
 senti_label
bullish    735
bearish    561
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60360 entries, 0 to 60359
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          60360 non-null  datetime64[ns]
 1   open          60360 non-null  float64       
 2   high          60360 non-null  float64       
 3   low           60360 non-null  float64       
 4   close         60360 non-null  float64       
 5   volume        60360 non-null  float64       
 6   tic           60360 non-null  object        
 7   day           60360 non-null  int32         
 8   macd          60360 non-null  float64       
 9   boll_ub       60360 non-null  float64       
 10  boll_lb       60360 non-null  float64       
 11  rsi_30        60360 non-null  float64       
 12  cci_30        60360 non-null  float64       
 13  dx_30         60360 non-null  float64       
 14  close_30_sma  60360 non-null  float64       
 15  close_60_sma  60360 non-null  float6

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_copy.to_csv('stocks_processed_data.csv', index=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
